In [1]:
import pandas as pd
import json
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from sklearn.tree import DecisionTreeClassifier
#import keras
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import torch
from transformers import BertTokenizer, BertModel
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
# 加载 JSON 数据
tweets_data = []
with open('tweets_DM.json', 'r') as file:
    for line in file:
        tweet = json.loads(line)
        # 提取嵌套字段 _source.tweet
        tweet_data = tweet.get('_source', {}).get('tweet', {})
        tweets_data.append(tweet_data)

# 转换为 DataFrame
tweets_df = pd.DataFrame(tweets_data)

# 查看结果
print(tweets_df.head())


                        hashtags  tweet_id  \
0                     [Snapchat]  0x376b20   
1  [freepress, TrumpLegacy, CNN]  0x2d5350   
2                   [bibleverse]  0x28b412   
3                             []  0x1cd5b0   
4                             []  0x2de201   

                                                text  
0  People who post "add me on #Snapchat" must be ...  
1  @brianklaas As we see, Trump is dangerous to #...  
2  Confident of your obedience, I write to you, k...  
3                Now ISSA is stalking Tasha 😂😂😂 <LH>  
4  "Trust is not the same as faith. A friend is s...  


In [3]:
# 加载 CSV 文件
emotion = pd.read_csv('emotion.csv')
data_identification = pd.read_csv('data_identification.csv')

# 合并数据：推文内容与情绪标签
tweets_with_emotion = pd.merge(tweets_df, emotion, on='tweet_id', how='left')

# 合并数据：训练/测试集标识
complete_data = pd.merge(tweets_with_emotion, data_identification, on='tweet_id', how='left')

# 查看结果
print(complete_data.head())

                        hashtags  tweet_id  \
0                     [Snapchat]  0x376b20   
1  [freepress, TrumpLegacy, CNN]  0x2d5350   
2                   [bibleverse]  0x28b412   
3                             []  0x1cd5b0   
4                             []  0x2de201   

                                                text       emotion  \
0  People who post "add me on #Snapchat" must be ...  anticipation   
1  @brianklaas As we see, Trump is dangerous to #...       sadness   
2  Confident of your obedience, I write to you, k...           NaN   
3                Now ISSA is stalking Tasha 😂😂😂 <LH>          fear   
4  "Trust is not the same as faith. A friend is s...           NaN   

  identification  
0          train  
1          train  
2           test  
3          train  
4           test  


In [4]:
# 数据清理
def clean_text(text):
    text = re.sub(r"http\S+", "", text)  # 去除URL
    text = re.sub(r"[^\w\s#@]", "", text)  # 保留文字、# 和 @
    return text.strip()

complete_data['text'] = complete_data['text'].apply(clean_text)
train_df = complete_data[complete_data['identification'] == 'train']
test_df = complete_data[complete_data['identification'] == 'test']

# BERT模型与分词器
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# 提取BERT嵌入
def extract_bert_features(texts, model, tokenizer, max_length=128):
    """
    将文本转换为BERT嵌入。
    """
    tokens = tokenizer(
        list(texts),
        max_length=max_length,
        truncation=True,
        padding='max_length',
        return_tensors='pt'
    )
    with torch.no_grad():
        outputs = model(**tokens)
        # 使用 [CLS] token 的嵌入作为句子向量
        cls_embeddings = outputs.last_hidden_state[:, 0, :]
    return cls_embeddings.numpy()

# 提取训练和测试集的特征
print("提取训练集特征...")
X_train = extract_bert_features(train_df['text'], bert_model, tokenizer)
y_train = train_df['emotion']

print("提取测试集特征...")
X_test = extract_bert_features(test_df['text'], bert_model, tokenizer)


# 构建逻辑回归模型
print("训练逻辑回归模型...")
clf = LogisticRegression(max_iter=1000, random_state=1)
clf.fit(X_train, y_train)

# 模型预测
print("预测测试集...")
y_pred = clf.predict(X_test)



# 保存预测结果
predictions = pd.DataFrame({
    'tweet_id': test_df['tweet_id'],
    'predicted_emotion': y_pred
})
predictions.to_csv('bert_predictions.csv', index=False)
print("预测结果保存到 bert_predictions.csv")

提取训练集特征...


RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 572350660608 bytes.

In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118